# VacationPy
---

## Maria's Code

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,273.51,64,95,11.79,TF,1697332081
1,1,wangdue phodrang,27.4862,89.8991,290.47,90,66,0.38,BT,1697332081
2,2,kailua-kona,19.6406,-155.9956,302.92,77,20,4.63,US,1697331999
3,3,la romana,18.4273,-68.9728,299.35,94,20,0.00,DO,1697332081
4,4,kingaroy,-26.5333,151.8333,302.98,19,0,2.20,AU,1697331823


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map plot
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
# Temps in KELVIN. 300K = 80F; 290K= 62F
hi_temp_filter_s = city_data_df["Max Temp"]<300
low_temp_filter_s = city_data_df["Max Temp"]>290
humidity_filter_s = city_data_df["Humidity"]<60
cloud_filter_s = city_data_df["Cloudiness"]<20

all_filter_s = hi_temp_filter_s & low_temp_filter_s & cloud_filter_s & humidity_filter_s
all_filter_s.sum()

31

In [39]:
ideal_city_data_df = city_data_df.loc[all_filter_s].dropna()

# Display sample data
print(ideal_city_data_df.shape)
ideal_city_data_df.head()

# Transform latitude and longitude to float
# ideal_city_data_df["Lat"] = ideal_city_data_df["Lat"].astype(float)
# ideal_city_data_df["Lng"] = ideal_city_data_df["Lng"].astype(float)


(31, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,christchurch,-43.5333,172.6333,292.82,53,1,5.66,NZ,1697332083
25,25,byron bay,-28.6500,153.6167,298.34,42,0,2.63,AU,1697331965
28,28,leteri,24.0500,77.4000,293.96,47,0,2.42,IN,1697332085
49,49,margaret river,-33.9500,115.0667,292.55,48,0,3.54,AU,1697332088
50,50,al bawiti,28.3492,28.8659,293.72,53,0,2.89,EG,1697332088


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# THERE'S a more EFFICIENT (less repetitive) way to do this!!!
#HOTEL_COLS = ["City", "Country", "Lat", "Lng", "Humidity"]
#hotel_df = ideal_city_data_df[HOTEL_COLS]


hotel_df = ideal_city_data_df.copy()
hotel_df = hotel_df.drop("Date", axis='columns')
hotel_df = hotel_df.drop("Wind Speed", axis='columns')
hotel_df = hotel_df.drop("Cloudiness", axis='columns')
hotel_df = hotel_df.drop("Max Temp", axis='columns')

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
14,14,christchurch,-43.5333,172.6333,53,NZ,
25,25,byron bay,-28.6500,153.6167,42,AU,
28,28,leteri,24.0500,77.4000,47,IN,
49,49,margaret river,-33.9500,115.0667,48,AU,
50,50,al bawiti,28.3492,28.8659,53,EG,
59,59,yanchep,-31.5500,115.6833,38,AU,
84,84,abomsa,9.9833,39.9833,50,ET,
114,114,alice springs,-23.7000,133.8833,12,AU,
126,126,bardai,11.6500,7.2333,35,NG,
138,138,al mutayrifi,25.4788,49.5576,40,SA,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "filter":" ",
    "limit":limit,
    "bias":" ",
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
christchurch - nearest hotel: Ibis Hotel Christchurch
byron bay - nearest hotel: Lord Byron Resort
leteri - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
al bawiti - nearest hotel: Old Oasis Hotel
yanchep - nearest hotel: Yanchep Inn
abomsa - nearest hotel: Kalid Hotel
alice springs - nearest hotel: Aurora Alice Springs
bardai - nearest hotel: No hotel found
al mutayrifi - nearest hotel: No hotel found
dubbo - nearest hotel: Blue Gum Motel
khairagarh - nearest hotel: No hotel found
banatski karlovac - nearest hotel: No hotel found
gallup - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
kilju - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
kiama - nearest hotel: Nova Kiama
crane - nearest hotel: No hotel found
as sanad - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
jahazpur - nearest hotel: No hotel found
turpan - nearest hotel: Oriental
ad 

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
14,14,christchurch,-43.5333,172.6333,53,NZ,Ibis Hotel Christchurch
25,25,byron bay,-28.6500,153.6167,42,AU,Lord Byron Resort
28,28,leteri,24.0500,77.4000,47,IN,No hotel found
49,49,margaret river,-33.9500,115.0667,48,AU,Margaret River Hotel
50,50,al bawiti,28.3492,28.8659,53,EG,Old Oasis Hotel
59,59,yanchep,-31.5500,115.6833,38,AU,Yanchep Inn
84,84,abomsa,9.9833,39.9833,50,ET,Kalid Hotel
114,114,alice springs,-23.7000,133.8833,12,AU,Aurora Alice Springs
126,126,bardai,11.6500,7.2333,35,NG,No hotel found
138,138,al mutayrifi,25.4788,49.5576,40,SA,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
map_plot

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)